In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
from datasets import load_dataset

paradigms = load_dataset("debate-land/2023-paradigms")["train"].train_test_split(test_size=0.3)

Found cached dataset csv (/Users/sam/.cache/huggingface/datasets/debate-land___csv/debate-land--2023-paradigms-4c55cafa927485ac/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
paradigms['train'][0]

{'text': 'Email: shoxha2020@gmail.com\nIntro / About Me:\nShout out to Westside High and UH - I wouldn\'t be anywhere without you. <3\nDon\'t be discriminatory. I\'m warning you now if you have to ask, "Is this problematic? Don\'t read it - there are better strategies out there.\nAlso Important: If you read spreading bad in front of me, I will not hack for you. I can spread and I can flow, but I am disabled and these skills were harder for me to develop than most. Many debaters see this as an opportunity for a persuasive 2ar and 2nr push, don\'t let this be you. I consider this motivated and ableist.\nYou\'re either winning an argument on the flow or you\'re not. Trivializing my struggles or the struggles of any judge for the ballot is an easy way to get me to despise you.\nDebate is a game, but it is an academic game. Tech over truth, but truth constrains tech. You\'ll have a harder time convincing me global warming is fake than convincing me warming will destroy the planet. If two de

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [28]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


In [29]:
tokenized_paradigms = paradigms.map(preprocess_function, batched=True)

Map:   0%|          | 0/343 [00:00<?, ? examples/s]

Map:   0%|          | 0/148 [00:00<?, ? examples/s]

In [30]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [31]:
import evaluate

accuracy = evaluate.load("accuracy")

In [32]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [33]:
id2label = {
    0: "FLOW",
    1: "FLAY",
    2: "LAY"
}

label2id = {
    "FLOW": 0,
    "FLAY": 1,
    "LAY": 2
}

In [34]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(tokenized_paradigms["train"]) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
#optimizer = tf.keras.optimizers.legacy.Adam()
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [35]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [36]:
tf_train_set: tf.data.Dataset = model.prepare_tf_dataset(
    tokenized_paradigms["train"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_paradigms["test"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [38]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [46]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="paradigm-model",
    tokenizer=tokenizer,
)

Cloning http-samc/paradigm-model into local empty directory.


OSError: WARNING: `git lfs clone` is deprecated and will not be updated
          with new flags from `git clone`

`git clone` has been updated in upstream Git to have comparable
speeds to `git lfs clone`.
fatal: repository 'http-samc/paradigm-model' does not exist
Error(s) during clone:
`git clone` failed: exit status 128


In [48]:
callbacks = [metric_callback]

In [49]:
import tensorflow as tf

model.compile(optimizer=optimizer)  # No loss argument!

In [50]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=callbacks)

In [51]:
model.export(filepath='model')

INFO:tensorflow:Assets written to: model/assets


In [156]:
text = """
I will vote with what's on what is on the flow only. I enter the round tabula rasa, i try to check my personal opinions at the door as best as i can. I may mock you for it, but I won’t vote against you for it. No paraphrasing. Quote the author, date and the exact words. Quals are even better but you don’t have to read them unless pressed. Have the website handy. Research is critical.

Speed? Meh. You cannot possibly go fast enough for me to not be able to follow you. However, that does not mean I want to hear you go fast. You can be quick and very persuasive. You don't need to spread.


"""
inputs = tokenizer(text, return_tensors="tf")

logits = model(**inputs).logits

predicted_class_id = int(tf.math.argmax(logits, axis=-1)[0])
model.config.id2label[predicted_class_id]

'LAY'

In [134]:
tf.math.argmax(logits, axis=-1)[0]
print("!!")

In [135]:
model.save_pretrained(save_directory='model')